- F, p, R^2 单个污染物与单个气象条件（13个）

- F, p单个污染物与所有气象条件

- 回归预测

- 计算AQI，首要污染物

- 对比：AQI对比

In [1]:
import os
# set working directory to root 
os.chdir('..')
os.getcwd()

'/home/qiangzibro/2021-math-model-code'

In [15]:
from common.io import load_table
import pandas as pd, numpy as np
from common.io import load, load_table
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import time
from threading import Thread
from multiprocessing import Process
import pandas as pd        
from pygam import LogisticGAM
from sklearn.datasets import load_breast_cancer

plt.style.use('ggplot')
# Linux上解决字体问题
plt.rcParams['font.sans-serif']= ['Noto Serif CJK JP'] #显示中文标签
plt.rcParams['axes.unicode_minus']=False 

In [71]:
def cal_time(func):
    def inner(args):
        start = time.time()
        result = func(args)
        end = time.time()
        print(f"{end-start}s passed")
        return result
    return inner

顺手写了一个多线程加载方法

In [72]:
# import concurrent

# @cal_time
# def load_df_threading():
#     df = {i:None for i in "ABC"}
#     def _load(p):
#         df[p] = load_table(p)
    
#     with concurrent.futures.ThreadPoolExecutor(max_workers=3) as executor:
#         futures = {executor.submit(_load, i): i for i in "ABC"}
#         for future in concurrent.futures.as_completed(futures):
#             p = futures[future]
#             try:
#                 df[p] = future.result()
#             except Exception as exc:
#                 print(exc)
#     return df

# data_df = load_df_threading()

In [73]:
import concurrent

@cal_time
def load(places):
    df = {i:None for i in places}
    with concurrent.futures.ProcessPoolExecutor(max_workers=len(places)) as executor:
        for p, result in zip(places, executor.map(load_table, places)):
            df[p] = result
    return df

data_df = load('ABC')

11.727554559707642s passed


In [18]:
@cal_time
def load_df():
    return  {
        "A":load_table("A"),
        "B":load_table("B"),
        "C":load_table("C"),
    }

data_df = load_df()

31.66095805168152s passed


In [69]:
cols = ['近地2米温度（℃）', '地表温度（K）',
       '比湿（kg/kg）', '湿度（%）', '近地10米风速（m/s）', '雨量（mm）', '云量',
       '边界层高度（m）', '大气压（Kpa）', '感热通量（W/m²）', '潜热通量（W/m²）', '长波辐射（W/m²）',
       '短波辐射（W/m²）']

In [70]:
pollutions = ['SO2小时平均浓度(μg/m³)', 'NO2小时平均浓度(μg/m³)',
       'PM10小时平均浓度(μg/m³)', 'PM2.5小时平均浓度(μg/m³)', 'O3小时平均浓度(μg/m³)',
       'CO小时平均浓度(mg/m³)']

In [80]:
df = data_df['A']['0']
X = df[cols]
y = df[pollutions[0]]
y = np.abs(y - y.mean())/(y.max() - y.min()) 
#Fit a model with the default parameters
gam = LogisticGAM().fit(X, y)

/usr/miniconda3/envs/math/lib/python3.7/site-packages/pygam/pygam.py:1112: RuntimeWarning: invalid value encountered in double_scalars
  r2['McFadden'] = full_ll/null_ll
/usr/miniconda3/envs/math/lib/python3.7/site-packages/pygam/pygam.py:1113: RuntimeWarning: invalid value encountered in double_scalars
  r2['McFadden_adj'] = 1. - (full_ll - self.statistics_['edof'])/null_ll


In [81]:
gam.summary()

LogisticGAM                                                                                               
=============================================== ==========================================================
Distribution:                      BinomialDist Effective DoF:                                    146.0044
Link Function:                        LogitLink Log Likelihood:                                       -inf
Number of Samples:                        25416 AIC:                                                   inf
                                                AICc:                                                  inf
                                                UBRE:                                               2.0392
                                                Scale:                                                 1.0
                                                Pseudo R-Squared:                                     0.22
Feature Function                  Lam

/usr/miniconda3/envs/math/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: KNOWN BUG: p-values computed in this summary are likely much smaller than they should be. 
 
Please do not make inferences based on these values! 

Collaborate on a solution, and stay up to date at: 
github.com/dswah/pyGAM/issues/163 

  """Entry point for launching an IPython kernel.


## Regression
For regression problems, we can use a linear GAM which models:

In [ ]:
from pygam import LinearGAM, s, f
from pygam.datasets import wage


X = df[cols]
y = df[pollutions[0]]
y = np.abs(y - y.mean())/(y.max() - y.min()) 
X = X.values

## model
gam = LinearGAM(s(0) + s(1) + f(2))
gam.gridsearch(X, y)

  0% (0 of 11) |                         | Elapsed Time: 0:00:00 ETA:  --:--:--

In [ ]:
## plotting
plt.figure();
fig, axs = plt.subplots(1,3);

titles = cols
for i, ax in enumerate(axs):
    XX = gam.generate_X_grid(term=i)
    ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX))
    ax.plot(XX[:, i], gam.partial_dependence(term=i, X=XX, width=.95)[1], c='r', ls='--')
    if i == 0:
        ax.set_ylim(-30,30)
    ax.set_title(titles[i]);

In [ ]:
from pygam import LinearGAM
from pygam.datasets import mcycle

X, y = mcycle(return_X_y=True)

gam = LinearGAM(n_splines=25).gridsearch(X, y)
XX = gam.generate_X_grid(term=0, n=500)

plt.plot(XX, gam.predict(XX), 'r--')
plt.plot(XX, gam.prediction_intervals(XX, width=.95), color='b', ls='--')

plt.scatter(X, y, facecolor='gray', edgecolors='none')
plt.title('95% prediction interval');

In [ ]:
# continuing last example with the mcycle dataset
for response in gam.sample(X, y, quantity='y', n_draws=50, sample_at_X=XX):
    plt.scatter(XX, response, alpha=.03, color='k')
plt.plot(XX, gam.predict(XX), 'r--')
plt.plot(XX, gam.prediction_intervals(XX, width=.95), color='b', ls='--')
plt.title('draw samples from the posterior of the coefficients')